# SQL introduktion

I det här avsnittet kollar vi hur man kan hämta ut data ur vårt Data warehouse med hjälp av SQL.

För att kunna köra nedanstående SQL-frågor behöver du antingen logga in på BigQuery, eller öppna denna Jupyter notebook i Google colab och köra följande kod: 


In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Vi har tidigare tittat på hur vi kan använda Stitchdata för att periodvis replikera över data från vårt Google Analytics-konto samt vår CRM-databas till BigQuery. Nedan tittar vi på hur man kan söka i dessa, samt hur man kan slå ihop data från Google Analytics och CRM-databasen.

## Select

Låt oss ta en snabb titt på vår Google Analytics-data genom att köra select-satsen nedan:

In [4]:
%%bigquery --project surfsapp-8011b

SELECT * FROM `surfsapp-8011b.google_analytics.report` LIMIT 5

,dimension1,end_date,sessions,_rjm_record_hash,_sdc_table_version,campaign,sourcemedium,_sdc_received_at,_sdc_sequence,start_date,transactions,pageviews,_sdc_batched_at,transactionrevenue
0,135193,2019-11-08 00:00:00+00:00,1,f1c2b319f8b8ad44c1eb6334052cb6eda95ab7cf73dd9e...,0,(not set),(direct) / (none),2019-11-23 04:12:04.977000+00:00,1574482324784,2019-11-08 00:00:00+00:00,0,24,2019-11-23 04:27:34.122000+00:00,0
1,135334,2019-11-08 00:00:00+00:00,1,dcd44ad1dd908ecd8671898c76a9eb225b5bd7bb5f2f26...,0,(not set),(direct) / (none),2019-11-23 04:12:04.977000+00:00,1574482324784,2019-11-08 00:00:00+00:00,0,6,2019-11-23 04:27:34.123000+00:00,0
2,135355,2019-11-08 00:00:00+00:00,1,908674783811bc690fc2347ee035705df3a3a10ce7d591...,0,(not set),(direct) / (none),2019-11-23 04:12:04.977000+00:00,1574482324784,2019-11-08 00:00:00+00:00,0,5,2019-11-23 04:27:34.123000+00:00,0
3,135365,2019-11-08 00:00:00+00:00,1,5dc302840a9ecb351ef0b17f81f0d8fca208dadc499084...,0,(not set),(direct) / (none),2019-11-23 04:12:04.977000+00:00,1574482324784,2019-11-08 00:00:00+00:00,0,14,2019-11-23 04:27:34.123000+00:00,0
4,135369,2019-11-08 00:00:00+00:00,1,e5b390c8fba5a892ee8d3e106b672ecf9cf80dc1b3b52b...,0,(not set),(direct) / (none),2019-11-23 04:12:04.977000+00:00,1574482324784,2019-11-08 00:00:00+00:00,0,9,2019-11-23 04:27:34.124000+00:00,0


Med select väljer vi ut vilka kolumner vi är intresserade av. Vi kan se att Stitchdata har lagt till en del kolumner som börjar på _rjm och _sdc som Stitchdata använder internt för att synka över data. För att göra vår dataanalys är vi inte intresserade av dessa utan väljer enbart de kolumner som vi är intresserade av:

In [5]:
%%bigquery --project surfsapp-8011b

SELECT distinct
  dimension1,
  start_date,
  campaign,
  sourcemedium,
  sessions,
  transactions,
  pageviews,
  transactionrevenue
FROM `surfsapp-8011b.google_analytics.report` LIMIT 5

,dimension1,start_date,campaign,sourcemedium,sessions,transactions,pageviews,transactionrevenue
0,135193,2019-11-08 00:00:00+00:00,(not set),(direct) / (none),1,0,24,0
1,135334,2019-11-08 00:00:00+00:00,(not set),(direct) / (none),1,0,6,0
2,135365,2019-11-08 00:00:00+00:00,(not set),(direct) / (none),1,0,14,0
3,135377,2019-11-08 00:00:00+00:00,(not set),(direct) / (none),1,0,12,0
4,135447,2019-11-08 00:00:00+00:00,(not set),(direct) / (none),1,0,8,0


I sökningen ovan har vi även lagt till *distinct* för att deduplicera resultaten. Anledningen till att vi behöver lägga till detta är att Stitchdata vid varje replikeringstillfälle tar med sig de senaste 15 dagarnas data och skickar denna till BigQuery och eftersom vi kör replikeringen var sjätte timma så kommer samma data att finnas på flera ställen i tabellen.

## Where

Med select väljer vi ut vilka kolumner vi vill läsa ut, men vill vi även specificera specifika rader som vi vill läsa ut så kan vi göra detta genom att lägga till en where-sats. Nedan tar vi bara med rader för ett specifikt datumintervall:


In [6]:
%%bigquery --project surfsapp-8011b

SELECT distinct
  dimension1,
  start_date,
  campaign,
  sourcemedium,
  sessions,
  transactions,
  pageviews,
  transactionrevenue
FROM `surfsapp-8011b.google_analytics.report` 
where start_date between '2019-11-06' and '2019-11-13' LIMIT 5

,dimension1,start_date,campaign,sourcemedium,sessions,transactions,pageviews,transactionrevenue
0,171569,2019-11-12 00:00:00+00:00,(not set),(direct) / (none),1,0,5,0
1,173517,2019-11-12 00:00:00+00:00,(not set),(direct) / (none),2,0,20,0
2,175035,2019-11-12 00:00:00+00:00,(not set),(direct) / (none),1,0,14,0
3,175755,2019-11-12 00:00:00+00:00,(not set),(direct) / (none),1,0,11,0
4,175917,2019-11-12 00:00:00+00:00,(not set),(direct) / (none),1,0,17,0


## Join

Det går som sagt att använda SQL, inte bara till att söka i data i en tabell, utan också för att kombinera och hämta ut data från flera tabeller.

Det som krävs för att man skall kunna slå ihop tabeller med hjälp av en join-sats är att det finns någon gemensam nyckel i båda tabellerna. I vår Google Analytics data ovan har vi t.ex. nyckeln *dimension1* som innehåller vårt user id. Tittar vi på vår CRM-data så kan vi se att vi motsvarande user id i en kolumn som heter *CustomerID*:

In [7]:
%%bigquery --project surfsapp-8011b

SELECT * FROM `surfsapp-8011b.nop_database.customer_data` LIMIT 5

,CustomerID,Gender,DateOfBirth,ZipPostalCode
0,138122,M,1958-08-03,20800
1,129472,M,1962-08-03,40010
2,135384,M,1962-08-03,40010
3,131694,F,1973-08-03,20800
4,140362,F,1973-08-03,11000


Om vi vill utöka vår GA-data med data om Gender, DateOfBirth, och ZipPostalCode från vårt CRM-system så kan vi göra det genom att utgå från vår GA-data och göra en left join med vår CRM-data där dimension1 i vår GA-data matchar CustomerID i vår CRM-data. 

In [9]:
%%bigquery --project surfsapp-8011b

SELECT *
FROM `surfsapp-8011b.google_analytics.ga_report` ga 
LEFT JOIN `surfsapp-8011b.nop_database.customer_data` crm ON ga.dimension1=CAST(crm.CustomerID AS string) LIMIT 5

,dimension1,start_date,campaign,sourcemedium,sessions,transactions,pageviews,transactionrevenue,CustomerID,Gender,DateOfBirth,ZipPostalCode
0,135468,2019-11-07 00:00:00+00:00,superdupersale,rix / radio,1,0,11,0,135468,M,1959-08-03,11000
1,174528,2019-11-11 00:00:00+00:00,superdupersale,rix / radio,1,1,19,30129,174528,M,1968-08-03,20800
2,169987,2019-11-12 00:00:00+00:00,superdupersale,rix / radio,1,1,17,25329,169987,M,1973-08-03,40010
3,167096,2019-11-07 00:00:00+00:00,superdupersale,rix / radio,2,1,25,45029,167096,M,1970-08-03,20800
4,135598,2019-11-07 00:00:00+00:00,superdupersale,rix / radio,2,1,29,4529,135598,F,1959-08-03,11000


Med left join behåller vi vår GA-data som den är och lägger till CRM-data på de rader där vi hittar ett matchande user-id i vår CRM-data. Hade vi istället gjort en inner join hade vi enbart tagit med user-id som finns i både vår GA-data och vår CRM-data och läst in data för dessa. Det går också att göra en outer join där vi tar med alla user-id som finns antingen i vår GA-data eller i vår CRM-data och hämtar den data vi har tillgänglig för detta ur respektive tabell...

## Group by

Vi kan också använda SQL för att aggregera data med hjälp av group by:


In [13]:
%%bigquery --project surfsapp-8011b

SELECT
  campaign,
  count(CustomerID) AS unique_userids,
  sum(sessions) AS sessions,
  sum(transactions) AS transactions,
  sum(transactionrevenue) AS revenue
FROM `surfsapp-8011b.google_analytics.ga_report` ga 
LEFT JOIN `surfsapp-8011b.nop_database.customer_data` crm ON ga.dimension1=CAST(crm.CustomerID AS string)
GROUP BY campaign

,campaign,unique_userids,sessions,transactions,revenue
0,(not set),1818,2115,524,9789796
1,black_friday,664,842,287,5975023
2,cyber_monday,332,351,35,390815
3,winter_getaways,50,54,7,52803
4,superdupersale,613,789,291,6332439


# Having

Slutligen kan vi även filtrera rader baserad på aggregerad information med having:

In [14]:
%%bigquery --project surfsapp-8011b

SELECT
  campaign,
  count(CustomerID) AS unique_userids,
  sum(sessions) AS sessions,
  sum(transactions) AS transactions,
  sum(transactionrevenue) AS revenue
FROM `surfsapp-8011b.google_analytics.ga_report` ga 
LEFT JOIN `surfsapp-8011b.nop_database.customer_data` crm ON ga.dimension1=CAST(crm.CustomerID AS string)
GROUP BY campaign
HAVING revenue>100000

,campaign,unique_userids,sessions,transactions,revenue
0,superdupersale,613,789,291,6332439
1,(not set),1818,2115,524,9789796
2,black_friday,664,842,287,5975023
3,cyber_monday,332,351,35,390815
